In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense

In [4]:
# path to the model weights files.
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 20000
nb_validation_samples = 4000
epochs = 50
batch_size = 16

# build the VGG16 network
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

model.summary()


Model loaded.
Found 20000 images belonging to 2 classes.
Found 3998 images belonging to 2 classes.
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                          

In [6]:
# fine-tune the model
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=2)


model.save('model_3.h5')  # creates a HDF5 file 'model_3.h5'

Epoch 1/50
1250/1250 - 124s - loss: 0.2836 - accuracy: 0.8767 - val_loss: 0.1989 - val_accuracy: 0.9205 - 124s/epoch - 99ms/step
Epoch 2/50
1250/1250 - 103s - loss: 0.1850 - accuracy: 0.9248 - val_loss: 0.1731 - val_accuracy: 0.9252 - 103s/epoch - 83ms/step
Epoch 3/50
1250/1250 - 106s - loss: 0.1547 - accuracy: 0.9383 - val_loss: 0.2034 - val_accuracy: 0.9277 - 106s/epoch - 85ms/step
Epoch 4/50
1250/1250 - 106s - loss: 0.1362 - accuracy: 0.9458 - val_loss: 0.1519 - val_accuracy: 0.9370 - 106s/epoch - 85ms/step
Epoch 5/50
1250/1250 - 105s - loss: 0.1193 - accuracy: 0.9507 - val_loss: 0.1581 - val_accuracy: 0.9417 - 105s/epoch - 84ms/step
Epoch 6/50
1250/1250 - 105s - loss: 0.1013 - accuracy: 0.9589 - val_loss: 0.1961 - val_accuracy: 0.9300 - 105s/epoch - 84ms/step
Epoch 7/50
1250/1250 - 106s - loss: 0.0947 - accuracy: 0.9628 - val_loss: 0.1475 - val_accuracy: 0.9422 - 106s/epoch - 85ms/step
Epoch 8/50
1250/1250 - 106s - loss: 0.0868 - accuracy: 0.9657 - val_loss: 0.1386 - val_accuracy: 